In [90]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [91]:
print('Build model...')
print('Load training data...')
df_x_train = pd.read_csv('X_train.csv', header=0, index_col = 0)
df_y_train = pd.read_csv('y_train.csv', header=0, index_col = 0)

x_train, x_test, y_train, y_test = train_test_split(df_x_train, df_y_train, test_size=0.2, random_state=0)

print(y_train)

Build model...
Load training data...
      y
id     
70    1
4347  0
1122  2
4570  1
34    1
...  ..
1033  1
3264  1
1653  1
2607  1
2732  1

[3840 rows x 1 columns]


In [92]:

print('Splitting into training and validation dataset')
X_train = x_train.values
Y_train = y_train['y'].values
X_val = x_test.values
Y_val = y_test['y'].values

print('Load testing data...')
df_x_test = pd.read_csv('X_test.csv', header=0, index_col = 0)
X_test = df_x_test.values

print(X_test)

Splitting into training and validation dataset
Load testing data...
[[ 0.39308068 -1.26658523 -0.53446496 ...  3.20821953 -0.05758947
   0.59172231]
 [ 0.97305006  0.12111063 -0.05152528 ...  0.92080134  0.13871761
  -0.05234103]
 [ 0.0382917  -0.99583852  0.08776399 ...  2.79359841  0.33395821
   0.79051858]
 ...
 [-0.55816281 -0.34474447 -0.17072247 ...  1.95796549  0.49549991
   1.33752358]
 [-0.43762359 -0.18208747  0.77121854 ...  0.51870441 -0.40840614
   0.87106538]
 [ 0.35841036  0.32877502  0.30282924 ...  0.50200468 -0.25083366
   0.21620879]]


In [93]:
# # encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(y)
# encoded_Y = encoder.transform(y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

# onehotencoder = OneHotEncoder(categorical_features = [0])
# x = onehotencoder.fit_transform(x).toarray()


In [110]:
# create model

model = Sequential()
model.add(Dense(3000, input_dim=1000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1500, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))
# Compile model

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'],
              weighted_metrics=['accuracy'])
earlier = EarlyStopping(monitor = 'val_accuracy_1',mode='min',min_delta=1e-3,patience=5,verbose=1)
checkpointer = ModelCheckpoint(filepath='best.hdf5',verbose=1, save_best_only=True, monitor='val_accuracy_1', mode='min')



In [111]:
cls_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), 
                                                Y_train)
cls_weight_dict = {0: cls_weights[0], 1: cls_weights[1]}
val_sample_weights = class_weight.compute_sample_weight(cls_weight_dict, Y_val)


In [112]:

model.fit(X_train, Y_train, epochs=10, batch_size=150, validation_data=(X_val, Y_val,val_sample_weights), callbacks=[earlier, checkpointer])



Train on 3840 samples, validate on 960 samples
Epoch 1/10
3750/3840 [============================>.] - ETA: 0s - loss: 0.7325 - accuracy: 0.7411 - accuracy_1: 0.7411
Epoch 00001: val_accuracy_1 improved from inf to 0.50213, saving model to best.hdf5
3840/3840 [==============================] - 6s 1ms/sample - loss: 0.7294 - accuracy: 0.7422 - accuracy_1: 0.7422 - val_loss: 0.7716 - val_accuracy: 0.7656 - val_accuracy_1: 0.5021
Epoch 2/10
3750/3840 [============================>.] - ETA: 0s - loss: 0.5253 - accuracy: 0.7875 - accuracy_1: 0.7875
Epoch 00002: val_accuracy_1 improved from 0.50213 to 0.50068, saving model to best.hdf5
3840/3840 [==============================] - 4s 1ms/sample - loss: 0.5217 - accuracy: 0.7885 - accuracy_1: 0.7885 - val_loss: 0.7471 - val_accuracy: 0.7760 - val_accuracy_1: 0.5007
Epoch 3/10
3750/3840 [============================>.] - ETA: 0s - loss: 0.4974 - accuracy: 0.7968 - accuracy_1: 0.7968
Epoch 00003: val_accuracy_1 did not improve from 0.50068
3840/

In [113]:
model.load_weights('best.hdf5')
Y_test = model.predict_classes(X_test)
print(Y_test)


f = open("best.csv", "w")
f.write("id,y\n")
for i,x in enumerate(Y_test):
    f.write("{},{}\n".format(i,x))
f.close()
        

[1 1 1 ... 1 1 1]


In [ ]:

from sklearn.metrics import balanced_accuracy_score
BMAC = balanced_accuracy_score(y_true, y_pred)